# list of activities to bve done in the job

1) Import Packages
2) Feature engineering already provided
3) Model loading and evaluation

In [19]:
#import packages

import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")

In [20]:
#loading the data
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

df = pd.read_csv("clean_data post EDA.csv")
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [21]:
df.head()

,Unnamed: 0,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,mean_year_price_off_peak_var,mean_year_price_peak_var,mean_year_price_mid_peak_var,mean_year_price_off_peak_fix,mean_year_price_peak_fix,mean_year_price_mid_peak_fix,mean_year_price_off_peak,mean_year_price_peak,mean_year_price_mid_peak,mean_6m_price_off_peak_var,mean_6m_price_peak_var,mean_6m_price_mid_peak_var,mean_6m_price_off_peak_fix,mean_6m_price_peak_fix,mean_6m_price_mid_peak_fix,mean_6m_price_off_peak,mean_6m_price_peak,mean_6m_price_mid_peak,mean_3m_price_off_peak_var,mean_3m_price_peak_var,mean_3m_price_mid_peak_var,mean_3m_price_off_peak_fix,mean_3m_price_peak_fix,mean_3m_price_mid_peak_fix,mean_3m_price_off_peak,mean_3m_price_peak,mean_3m_price_mid_peak,churn
0,0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,0.124787,0.100749,0.066530,40.942265,22.352010,14.901340,41.067053,22.452760,14.967871,0.123598,0.098104,0.061525,41.318559,20.364442,13.576296,41.442157,20.462545,13.637821,0.131756,0.092638,0.036909,42.497907,12.218665,8.145777,42.629663,12.311304,8.182687,1
1,1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0.149609,0.007124,0.000000,44.311375,0.000000,0.000000,44.460984,0.007124,0.000000,0.147851,0.014247,0.000000,44.355820,0.000000,0.000000,44.503671,0.014247,0.000000,0.147600,0.000000,0.000000,44.444710,0.000000,0.000000,44.592310,0.000000,0.000000,0
2,2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0.170512,0.088421,0.000000,44.385450,0.000000,0.000000,44.555962,0.088421,0.000000,0.168569,0.088535,0.000000,44.444710,0.000000,0.000000,44.613279,0.088535,0.000000,0.167798,0.088409,0.000000,44.444710,0.000000,0.000000,44.612508,0.088409,0.000000,0
3,3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0.151210,0.000000,0.000000,44.400265,0.000000,0.000000,44.551475,0.000000,0.000000,0.149330,0.000000,0.000000,44.444710,0.000000,0.000000,44.594040,0.000000,0.000000,0.148586,0.000000,0.000000,44.444710,0.000000,0.000000,44.593296,0.000000,0.000000,0
4,4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0.124174,0.103638,0.072865,40.688156,24.412893,16.275263,40.812330,24.516531,16.348129,0.121266,0.102368,0.073728,40.728885,24.437330,16.291555,40.850151,24.539698,16.365283,0.119906,0.101673,0.073719,40.728885,24.437330,16.291555,40.848791,24.539003,16.365274,0


In [22]:
price_df = pd.read_csv("price_data.csv")
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [23]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()



In [24]:
monthly_price_by_id.head()

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,40.565969
1,0002203ffbb812588b632b9e628cc38d,2015-02-01,0.126098,40.565969
2,0002203ffbb812588b632b9e628cc38d,2015-03-01,0.128067,40.728885
3,0002203ffbb812588b632b9e628cc38d,2015-04-01,0.128067,40.728885
4,0002203ffbb812588b632b9e628cc38d,2015-05-01,0.128067,40.728885


In [25]:
# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()



In [26]:
jan_prices.head()

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,40.565969
1,0004351ebdd665e6ee664792efc4fd13,2015-01-01,0.148047,44.266931
2,0010bcc39e42b3c2131ed2ce55246e3c,2015-01-01,0.150837,44.444710
3,0010ee3855fdea87602a5b7aba8e42de,2015-01-01,0.123086,40.565969
4,00114d74e963e47177db89bc70108537,2015-01-01,0.149434,44.266931


In [27]:
dec_prices.head()

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-12-01,0.119906,40.728885
1,0004351ebdd665e6ee664792efc4fd13,2015-12-01,0.143943,44.444710
2,0010bcc39e42b3c2131ed2ce55246e3c,2015-12-01,0.201280,45.944710
3,0010ee3855fdea87602a5b7aba8e42de,2015-12-01,0.113068,40.728885
4,00114d74e963e47177db89bc70108537,2015-12-01,0.145440,44.266930


In [28]:
# Calculate the difference
jan_prices = jan_prices.rename(columns={'price_off_peak_var': 'jan1', 'price_off_peak_fix': 'jan2'})
dec_prices = dec_prices.rename(columns={'price_off_peak_var':'dec1','price_off_peak_fix':'dec2'})

diff = pd.merge(dec_prices,jan_prices.drop(columns='price_date'),on='id')


In [29]:
diff.head()

,id,price_date,dec1,dec2,jan1,jan2
0,0002203ffbb812588b632b9e628cc38d,2015-12-01,0.119906,40.728885,0.126098,40.565969
1,0004351ebdd665e6ee664792efc4fd13,2015-12-01,0.143943,44.444710,0.148047,44.266931
2,0010bcc39e42b3c2131ed2ce55246e3c,2015-12-01,0.201280,45.944710,0.150837,44.444710
3,0010ee3855fdea87602a5b7aba8e42de,2015-12-01,0.113068,40.728885,0.123086,40.565969
4,00114d74e963e47177db89bc70108537,2015-12-01,0.145440,44.266930,0.149434,44.266931


In [30]:
diff['offpeak_diff_dec_january_energy'] = diff['dec1'] - diff['jan1']
diff['offpeak_diff_dec_january_power'] = diff['dec2'] - diff['jan2']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]

In [31]:
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [32]:
# merging the client_df with diff

final_df = pd.merge(df,diff, on='id',how='inner')



In [33]:
final_df = final_df.drop('Unnamed: 0',axis=1)

In [34]:

final_df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,mean_year_price_off_peak_var,mean_year_price_peak_var,mean_year_price_mid_peak_var,mean_year_price_off_peak_fix,mean_year_price_peak_fix,mean_year_price_mid_peak_fix,mean_year_price_off_peak,mean_year_price_peak,mean_year_price_mid_peak,mean_6m_price_off_peak_var,mean_6m_price_peak_var,mean_6m_price_mid_peak_var,mean_6m_price_off_peak_fix,mean_6m_price_peak_fix,mean_6m_price_mid_peak_fix,mean_6m_price_off_peak,mean_6m_price_peak,mean_6m_price_mid_peak,mean_3m_price_off_peak_var,mean_3m_price_peak_var,mean_3m_price_mid_peak_var,mean_3m_price_off_peak_fix,mean_3m_price_peak_fix,mean_3m_price_mid_peak_fix,mean_3m_price_off_peak,mean_3m_price_peak,mean_3m_price_mid_peak,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,0.124787,0.100749,0.066530,40.942265,22.352010,14.901340,41.067053,22.452760,14.967871,0.123598,0.098104,0.061525,41.318559,20.364442,13.576296,41.442157,20.462545,13.637821,0.131756,0.092638,0.036909,42.497907,12.218665,8.145777,42.629663,12.311304,8.182687,1,0.020057,3.700961
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0.149609,0.007124,0.000000,44.311375,0.000000,0.000000,44.460984,0.007124,0.000000,0.147851,0.014247,0.000000,44.355820,0.000000,0.000000,44.503671,0.014247,0.000000,0.147600,0.000000,0.000000,44.444710,0.000000,0.000000,44.592310,0.000000,0.000000,0,-0.003767,0.177779
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0.170512,0.088421,0.000000,44.385450,0.000000,0.000000,44.555962,0.088421,0.000000,0.168569,0.088535,0.000000,44.444710,0.000000,0.000000,44.613279,0.088535,0.000000,0.167798,0.088409,0.000000,44.444710,0.000000,0.000000,44.612508,0.088409,0.000000,0,-0.004670,0.177779
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0.151210,0.000000,0.000000,44.400265,0.000000,0.000000,44.551475,0.000000,0.000000,0.149330,0.000000,0.000000,44.444710,0.000000,0.000000,44.594040,0.000000,0.000000,0.148586,0.000000,0.000000,44.444710,0.000000,0.000000,44.593296,0.000000,0.000000,0,-0.004547,0.177779
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0.124174,0.103638,0.072865,40.688156,24.412893,16.275263,40.812330,24.516531,16.348129,0.121266,0.102368,0.073728,40.728885,24.437330,16.291555,40.850151,24.539698,16.365283,0.119906,0.101673,0.073719,40.728885,24.437330,16.291555,40.848791,24.539003,16.365274,0,-0.006192,0.162916


In [35]:
# rerrranging the columns

df3 = final_df[[col for col in final_df.columns if col != 'churn'] + ['churn']]
df3.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,mean_year_price_off_peak_var,mean_year_price_peak_var,mean_year_price_mid_peak_var,mean_year_price_off_peak_fix,mean_year_price_peak_fix,mean_year_price_mid_peak_fix,mean_year_price_off_peak,mean_year_price_peak,mean_year_price_mid_peak,mean_6m_price_off_peak_var,mean_6m_price_peak_var,mean_6m_price_mid_peak_var,mean_6m_price_off_peak_fix,mean_6m_price_peak_fix,mean_6m_price_mid_peak_fix,mean_6m_price_off_peak,mean_6m_price_peak,mean_6m_price_mid_peak,mean_3m_price_off_peak_var,mean_3m_price_peak_var,mean_3m_price_mid_peak_var,mean_3m_price_off_peak_fix,mean_3m_price_peak_fix,mean_3m_price_mid_peak_fix,mean_3m_price_off_peak,mean_3m_price_peak,mean_3m_price_mid_peak,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,0.124787,0.100749,0.066530,40.942265,22.352010,14.901340,41.067053,22.452760,14.967871,0.123598,0.098104,0.061525,41.318559,20.364442,13.576296,41.442157,20.462545,13.637821,0.131756,0.092638,0.036909,42.497907,12.218665,8.145777,42.629663,12.311304,8.182687,0.020057,3.700961,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0.149609,0.007124,0.000000,44.311375,0.000000,0.000000,44.460984,0.007124,0.000000,0.147851,0.014247,0.000000,44.355820,0.000000,0.000000,44.503671,0.014247,0.000000,0.147600,0.000000,0.000000,44.444710,0.000000,0.000000,44.592310,0.000000,0.000000,-0.003767,0.177779,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0.170512,0.088421,0.000000,44.385450,0.000000,0.000000,44.555962,0.088421,0.000000,0.168569,0.088535,0.000000,44.444710,0.000000,0.000000,44.613279,0.088535,0.000000,0.167798,0.088409,0.000000,44.444710,0.000000,0.000000,44.612508,0.088409,0.000000,-0.004670,0.177779,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0.151210,0.000000,0.000000,44.400265,0.000000,0.000000,44.551475,0.000000,0.000000,0.149330,0.000000,0.000000,44.444710,0.000000,0.000000,44.594040,0.000000,0.000000,0.148586,0.000000,0.000000,44.444710,0.000000,0.000000,44.593296,0.000000,0.000000,-0.004547,0.177779,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0.124174,0.103638,0.072865,40.688156,24.412893,16.275263,40.812330,24.516531,16.348129,0.121266,0.102368,0.073728,40.728885,24.437330,16.291555,40.850151,24.539698,16.365283,0.119906,0.101673,0.073719,40.728885,24.437330,16.291555,40.848791,24.539003,16.365274,-0.006192,0.162916,0


# Advantages of using Random forest for churn prediction in this case
    1)Accuracy: Random Forest generally provides high prediction accuracy compared to other algorithms. It is capable of capturing complex relationships and interactions between features, making it suitable for churn prediction where multiple factors may contribute to customer churn.

    2)Robust to Overfitting: Random Forest is less prone to overfitting compared to individual decision trees. The ensemble nature of Random Forest, which combines multiple decision trees, helps to reduce overfitting and improve generalization.

    3)Feature Importance: Random Forest can provide insights into feature importance, indicating which features have the most significant impact on predicting churn. This information can be valuable for understanding the underlying factors driving customer churn.

    4)Handling of Missing Data: Random Forest can effectively handle missing data in the dataset. It can handle missing values without imputing them, which simplifies the preprocessing step and saves time.

    5)Robustness to Outliers: Random Forest is relatively robust to outliers and noise in the data. Outliers have less impact on the overall model performance due to the averaging effect of multiple trees.


# Disadvantages of using Random Forest for churn prediction in this use case:

    1)Model Interpretability: Random Forest models are not as easily interpretable as simpler models like logistic regression. Understanding the exact reasoning behind individual predictions can be challenging, especially when dealing with a large number of trees in the ensemble.

    2)Computationally Expensive: Training a Random Forest model can be computationally expensive, particularly when dealing with a large number of features and a substantial amount of data. It requires more computational resources and time compared to simpler models.

    3)Hyperparameter Tuning: Random Forest has several hyperparameters that need to be tuned to achieve optimal performance. Finding the right set of hyperparameters can be time-consuming and may require extensive experimentation.

    4)Imbalanced Data: Random Forest can face challenges in handling imbalanced datasets, where the number of churned customers is significantly lower than non-churned customers. Special techniques such as resampling or class weighting may be necessary to address this issue.

    5)Model Size: Random Forest models can be relatively large in size, especially when dealing with a large number of trees. This can be a consideration when deploying the model in production systems with limited memory or storage resources

In [36]:
#importing the models and other necessary features required

from sklearn.model_selection import train_test_split ,cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchC


ImportError: cannot import name 'GridSearchC' from 'sklearn.model_selection' (c:\Users\Dipayan Bhowal\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\__init__.py)

In [ ]:
categorical_channels = ['channel_sales','has_gas','origin_up']

encoder = LabelEncoder()

for col in categorical_channels:
    try:
        df3[col]= encoder.fit_transform(df3[col])
        encoded_labels = df3['channel_sales'].unique()
    except ValueError as e:
        print(e)

# Here we are using label encoders to  encode the categorical variables in the data

In [ ]:
#making the train and test features

df3 = df3.drop(columns='id',axis=1)

In [ ]:
timestamp_columns = df3.select_dtypes(include=['datetime64']).columns


for col in timestamp_columns:
    
    df3[col] = df3[col].apply(lambda x: x.timestamp())

In [ ]:
df3.dtypes

channel_sales                        int32
cons_12m                             int64
cons_gas_12m                         int64
cons_last_month                      int64
date_activ                         float64
date_end                           float64
date_modif_prod                    float64
date_renewal                       float64
forecast_cons_12m                  float64
forecast_cons_year                   int64
forecast_discount_energy           float64
forecast_meter_rent_12m            float64
forecast_price_energy_off_peak     float64
forecast_price_energy_peak         float64
forecast_price_pow_off_peak        float64
has_gas                              int32
imp_cons                           float64
margin_gross_pow_ele               float64
margin_net_pow_ele                 float64
nb_prod_act                          int64
net_margin                         float64
num_years_antig                      int64
origin_up                            int32
pow_max    

In [ ]:
df3.columns

Index(['channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'mean_year_price_off_peak_var', 'mean_year_price_peak_var',
       'mean_year_price_mid_peak_var', 'mean_year_price_off_peak_fix',
       'mean_year_price_peak_fix', 'mean_year_price_mid_peak_fix',
       'mean_year_price_off_peak', 'mean_year_price_peak',
       'mean_year_price_mid_peak', 'mean_6m_price_off_peak_var',
       'mean_6m_price_peak_var', 'mean_6m_price_mid_peak_var',
       'mean_6m_price_off_peak_fix', 'mean_6m_price_peak_fix',
       'mean_6m_price_mid_peak_fix', 

In [ ]:
# preparing the model 

X = df3.drop(columns='churn',axis=1)
Y = df3[['churn']]

X_train ,X_test , Y_train , Y_test = train_test_split(X,Y,stratify=Y,test_size=0.3,random_state=3)
    
  

In [ ]:
X_train.head()

,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,mean_year_price_off_peak_var,mean_year_price_peak_var,mean_year_price_mid_peak_var,mean_year_price_off_peak_fix,mean_year_price_peak_fix,mean_year_price_mid_peak_fix,mean_year_price_off_peak,mean_year_price_peak,mean_year_price_mid_peak,mean_6m_price_off_peak_var,mean_6m_price_peak_var,mean_6m_price_mid_peak_var,mean_6m_price_off_peak_fix,mean_6m_price_peak_fix,mean_6m_price_mid_peak_fix,mean_6m_price_off_peak,mean_6m_price_peak,mean_6m_price_mid_peak,mean_3m_price_off_peak_var,mean_3m_price_peak_var,mean_3m_price_mid_peak_var,mean_3m_price_off_peak_fix,mean_3m_price_peak_fix,mean_3m_price_mid_peak_fix,mean_3m_price_off_peak,mean_3m_price_peak,mean_3m_price_mid_peak,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
10994,4,8145,0,1679,1.344384e+09,1.470614e+09,1.434499e+09,1.439078e+09,1201.33,1679,0.0,16.79,0.141666,0.000000,44.311378,0,240.46,16.44,16.44,1,112.37,3,3,13.856,0.147423,0.00000,0.000000,44.370635,0.000000,0.00000,44.518059,0.000000,0.000000,0.144250,0.000000,0.000000,44.444710,0.000000,0.000000,44.588960,0.000000,0.000000,0.143538,0.000000,0.000000,44.444710,0.00000,0.000000,44.588248,0.000000,0.000000,-0.007829,0.177779
13433,4,10511,0,0,1.344557e+09,1.470787e+09,1.434499e+09,1.439251e+09,1531.64,0,0.0,15.24,0.141666,0.000000,44.311378,0,0.00,13.92,13.92,1,128.91,3,4,13.100,0.147273,0.00000,0.000000,44.326190,0.000000,0.00000,44.473464,0.000000,0.000000,0.144050,0.000000,0.000000,44.385450,0.000000,0.000000,44.529500,0.000000,0.000000,0.143538,0.000000,0.000000,44.444710,0.00000,0.000000,44.588248,0.000000,0.000000,-0.007829,0.177779
5177,7,11824,0,76,1.285546e+09,1.474934e+09,1.440634e+09,1.443398e+09,1251.71,76,0.0,141.71,0.115360,0.100243,40.606701,0,7.51,13.78,13.78,1,143.33,5,3,20.000,0.125525,0.10546,0.075275,40.633851,24.380309,16.25354,40.759377,24.485769,16.328815,0.121178,0.103183,0.074754,40.701733,24.421038,16.280693,40.822911,24.524222,16.355446,0.118353,0.101898,0.074321,40.728885,24.43733,16.291555,40.847238,24.539228,16.365876,-0.012679,0.162916
5160,5,101588,39114,3635,1.361750e+09,1.456358e+09,1.450310e+09,1.424909e+09,1840.04,0,0.0,16.30,0.140912,0.000000,44.311378,1,0.00,31.20,31.20,3,156.63,3,2,13.856,0.145808,0.00000,0.000000,44.266930,0.000000,0.00000,44.412739,0.000000,0.000000,0.143720,0.000000,0.000000,44.266930,0.000000,0.000000,44.410650,0.000000,0.000000,0.142211,0.000000,0.000000,44.266930,0.00000,0.000000,44.409141,0.000000,0.000000,-0.005836,-0.000001
7635,4,2292,4967,0,1.323907e+09,1.481760e+09,1.446336e+09,1.450224e+09,341.81,0,0.0,15.33,0.143460,0.000000,45.806878,1,0.00,16.14,16.14,3,37.73,4,4,10.392,0.148707,0.00000,0.000000,44.311375,0.000000,0.00000,44.460083,0.000000,0.000000,0.146132,0.000000,0.000000,44.355820,0.000000,0.000000,44.501952,0.000000,0.000000,0.144524,0.000000,0.000000,44.444710,0.00000,0.000000,44.589234,0.000000,0.000000,-0.007921,0.177779


# as the model we are using is random forest so we are not using any standardization and randomization of the values
    1)  The hyperparameter tuning of the Random forest is done with params
    2) The Business Requiremnt asks for a classification(whether customer is churn or retained) so we are using the RandomForest Classifier

In [ ]:





model = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth':[None, 5, 10],
    'min_samples_split': [2, 5, 10]
    

}
try:
    grid_search = GridSearchCV(estimator=model,param_grid=param_grid,cv=5)

    grid_search.fit(X_train,Y_train)
    print("The Best parameters:",grid_search.best_params_)
    print("Best_Score:",grid_search.best_score_)
except Exception as e:
    print(e)

The Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 300}
Best_Score: 0.9069743395901415


# The best parameters for the Hyperparameter of the classifier
    1)The Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 300}

    2)this parameter for the model of random forest generates the highest accuracy

In [ ]:
# So plotting the best of Random Forest on the data

Classifier = RandomForestClassifier(max_depth=None, min_samples_split=2, n_estimators=300)

#fitting the training data 

Classifier.fit(X_train,Y_train)

accuracy = Classifier.score(X_test, Y_test)
print("Accuracy:", accuracy)


Accuracy: 0.9057507987220448


In [ ]:
Y_pred = Classifier.predict(X_test)
accuracy = accuracy_score(Y_pred,Y_test)
accuracy_percentage = accuracy *100

print("The accuracy of the model is:{:.2f}%".format(accuracy_percentage) )

The accuracy of the model is:90.58%


# is my model performance is satisfactory? 

    Comparison to Baseline: It is essential to compare the model's accuracy to a baseline or random chance. If the baseline accuracy is significantly lower than 90.56%, then achieving such high accuracy demonstrates the effectiveness of the model. However, if the baseline accuracy is already close to or higher than 90.58%, it suggests that the model's performance might not be as satisfactory.

    Business Context: The definition of "satisfactory" depends on the specific requirements and objectives of the business. Consider the impact of misclassifications and the associated costs. If the consequences of misidentifying a customer at risk of churn are severe, then a higher level of accuracy might be necessary for satisfactory performance. On the other hand, if the cost of false positives or false negatives is relatively low, a 90.56% accuracy might be considered satisfactory.

    Trade-Offs: Model performance should be evaluated in conjunction with other factors such as computational resources, data availability, and model complexity. If achieving a higher accuracy requires significant computational resources or complex models that are difficult to interpret or maintain, then the 90.58% accuracy may be satisfactory given the trade-offs involved.

    Benchmarking: Compare the model's accuracy to similar industry standards or other published research in the field. If the model's accuracy is on par with or higher than existing benchmarks, it indicates a satisfactory performance.

    Validation Metrics: Accuracy alone might not be sufficient to assess model performance. Consider other evaluation metrics such as precision, recall, F1 score, or ROC AUC to gain a more comprehensive understanding of the model's predictive capability.

In [ ]:
import pickle

with open('churn_prediction_model.pkl', 'wb') as f:
    pickle.dump(Classifier, f)

In [ ]:
#loading the trained churn prediction model
with open("churn_prediction_model.pkl", "rb") as file:
    model = pickle.load(file)

In [ ]:
selected_features = [
    
    'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'mean_year_price_off_peak_var', 'mean_year_price_peak_var',
       'mean_year_price_mid_peak_var', 'mean_year_price_off_peak_fix',
       'mean_year_price_peak_fix', 'mean_year_price_mid_peak_fix',
       'mean_year_price_off_peak', 'mean_year_price_peak',
       'mean_year_price_mid_peak', 'mean_6m_price_off_peak_var',
       'mean_6m_price_peak_var', 'mean_6m_price_mid_peak_var',
       'mean_6m_price_off_peak_fix', 'mean_6m_price_peak_fix',
       'mean_6m_price_mid_peak_fix', 'mean_6m_price_off_peak',
       'mean_6m_price_peak', 'mean_6m_price_mid_peak',
       'mean_3m_price_off_peak_var', 'mean_3m_price_peak_var',
       'mean_3m_price_mid_peak_var', 'mean_3m_price_off_peak_fix',
       'mean_3m_price_peak_fix', 'mean_3m_price_mid_peak_fix',
       'mean_3m_price_off_peak', 'mean_3m_price_peak',
       'mean_3m_price_mid_peak', 'offpeak_diff_dec_january_energy',
       'offpeak_diff_dec_january_power',
]


In [ ]:
prediction_input = df3[selected_features]

In [ ]:
churn_probabilities = model.predict_proba(prediction_input)[:, 1] 

In [ ]:
#Determine the threshold for identifying customers on the brink of churn
threshold = 0.7  
brink_of_churn_customers = df3[churn_probabilities > threshold]
#  Apply discount or retention strategy to retain these customers
#  Apply a discount of 20% to the "mean_year_price_off_peak_fix" feature for the customers on the brink of churn
brink_of_churn_customers['mean_year_price_off_peak_fix'] = brink_of_churn_customers['mean_year_price_off_peak_fix'] * 0.8
#  Count the number of customers on the brink of churn
num_customers_on_brink = len(brink_of_churn_customers)

#  Print the number of customers on the brink of churn
print("Number of customers on the brink of churn:", num_customers_on_brink)

Number of customers on the brink of churn: 421


In [ ]:
df3["Churn_probability"] = churn_probabilities

In [ ]:
off_peak_energy_price = 0.10  # Placeholder value for off-peak energy price in currency per unit (e.g., per kWh)
peak_energy_price = 0.15  # Placeholder value for peak energy price in currency per unit (e.g., per kWh)
off_peak_power_price = 0.08  # Placeholder value for off-peak power price in currency per unit (e.g., per kW)
df3['revenue'] = 0  # Initialize the 'revenue' column with zeros

# Calculate revenue based on consumption and pricing
df3['revenue'] += df3['cons_12m'] * off_peak_energy_price  # Revenue from off-peak energy consumption
df3['revenue'] += df3['cons_last_month'] * peak_energy_price  # Revenue from peak energy consumption
df3['revenue'] += df3['imp_cons'] * off_peak_power_price  # Revenue from off-peak power consumption

# Additional revenue calculations based on other features, such as forecasted consumption, discount, etc.

# Print the revenue column
print(df3['revenue'])


0             0.0000
1           466.0000
2            54.4000
3           158.4000
4           525.5856
5          1144.3968
6          4509.7000
7          3149.8504
8           296.2000
9          2952.1672
10         1577.8584
11          842.2228
12         5445.9944
13          772.6000
14          970.7000
15         5188.7324
16        91742.8804
17         3077.6000
18         2921.6988
19         1541.1000
20         1979.5832
21         3346.6444
22         2670.5112
23         1215.7000
24         1312.2000
25          409.3088
26          263.2000
27          479.2000
28          969.1056
29       210348.8500
30           32.4000
31          210.3000
32         1959.3000
33         4838.2532
34         4334.3740
35         4680.2664
36       234076.0132
37         2042.9000
38          718.4936
39          604.7000
40       240782.9424
41         1158.8512
42          425.9000
43         1219.9000
44         1385.9000
45        12207.1144
46         8422.9620
47          5

In [ ]:
# Calculate potential revenue loss without discounts
revenue_loss = (df3['Churn_probability'] * df3['revenue']).sum()

# Print the estimated revenue loss
print("Estimated revenue loss without discounts: $", revenue_loss)





Estimated revenue loss without discounts: $ 19930646.397034667


# How much money could a client save with the use of the model? What assumptions did i make to come to this conclusion?
    
    
    1)Identify the customers on the brink of churn: Use the trained model to predict the churn probabilities for each customer in the dataset. Set a threshold to identify customers who are considered on the brink of churn (e.g., churn probability above 0.5).

    2)Apply discounts to retain customers: Once the customers on the brink of churn are identified, apply discounts to relevant features or services to incentivize them to stay. For example, you can apply a discount to the mean_year_price_off_peak_fix feature for those customers.

    3) cost savings: To estimate the cost savings, you need to make assumptions about the discount rate, customer response to discounts, and the potential revenue loss due to churn. Here's a general approach:

        a. Assume a discount rate: Determine the percentage discount you apply to the relevant feature. For example, if you apply a 20% discount, assume the discounted price is 80% of the original price.

        b. Assume customer response: Estimate the percentage of customers who will be retained due to the applied discount. This can vary based on factors like the attractiveness of the discount, customer loyalty, and the overall quality of service.

        c. Estimate revenue loss without discounts: Calculate the potential revenue loss if the customers on the brink of churn actually churned without any applied discounts. This can be done by multiplying the churn probability of each customer by their corresponding revenue.

        d. Estimate revenue loss with discounts: Calculate the potential revenue loss considering the discounts and assumed customer retention rate. Multiply the churn probability of each customer by the discounted revenue (original revenue * discount rate * customer retention rate).

        e. Calculate cost savings: Subtract the estimated revenue loss with discounts from the estimated revenue loss without discounts to determine the potential cost savings.